In [21]:
from scipy.stats import entropy
import numpy as np
import torch
import torch.nn as nn
import huffman
import sys

In [38]:
len(' (huffman):')

11

24

In [25]:
sys.getsizeof(all_weights[0])

80

In [24]:
import sys

# Exemplo com diferentes tipos de variáveis
a = 42  # Inteiro
b = "H"  # String
c = [1, 2, 3, 4, 5]  # Lista
d = {"chave": "valor"}  # Dicionário

# Obtendo os tamanhos
print(sys.getsizeof(a))  # Tamanho do inteiro
print(sys.getsizeof(b))  # Tamanho da string
print(sys.getsizeof(c))  # Tamanho da lista
print(sys.getsizeof(d))  # Tamanho do dicionário


28
50
120
232


In [43]:
def entropy(probabilidades):
    return -np.sum(probabilidades * np.log2(probabilidades))

In [6]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(3, 4)
    self.fc2 = nn.Linear(4, 1)
  
  def forward(self, x):
    x = self.fc1(x)
    return self.fc2(x)

In [10]:
model = Model()
for name, param in model.named_parameters():
    if 'weight' in name:
        print(param)

Parameter containing:
tensor([[-0.4100, -0.1536, -0.3355],
        [ 0.3944, -0.4970, -0.3327],
        [-0.5150, -0.2089, -0.4499],
        [-0.4587,  0.3772, -0.3397]], requires_grad=True)
Parameter containing:
tensor([[ 0.0670,  0.3719, -0.1998, -0.3650]], requires_grad=True)


In [41]:
import numpy as np
from scipy.stats import entropy

# Dados de exemplo: todos os valores são 1
data = np.random.rand(100000)

# Calculando a entropia
# Primeiro, calculamos a distribuição de frequência dos dados
hist, bin_edges = np.histogram(data, bins=np.unique(data), density=True)

# Agora, calculamos a entropia
ent = entropy(hist)
print("Entropia:", ent)


Entropia: 6.993609373537418


In [38]:
bin_edges

array([1.])

In [1]:
import torch.nn as nn
import torch

In [43]:
param = nn.Linear(1, 4)
weights = param.weight
prob = torch.abs(weights) / torch.sum(torch.abs(weights))
prob

tensor([[0.3268],
        [0.1880],
        [0.2387],
        [0.2465]], grad_fn=<DivBackward0>)

tensor([[0.3254],
        [0.3621],
        [0.2353],
        [0.0773]], grad_fn=<DivBackward0>)